In [129]:
import numpy as np
from random import uniform
from matplotlib.patches import Ellipse
from scipy.stats import multivariate_normal
from scipy.sparse import diags

In [130]:
from sklearn.cluster import KMeans
class GMM:
    """
    Full covariance Gaussian Mixture Model,
    trained using Expectation Maximization.

    Parameters
    ----------
    n_components : int
        Number of clusters/mixture components in which the data will be
        partitioned into.

    n_iters : int
        Maximum number of iterations to run the algorithm.

    tol : float
        Tolerance. If the log-likelihood between two iterations is smaller than
        the specified tolerance level, the algorithm will stop performing the
        EM optimization.

    seed : int
        Seed / random state used to initialize the parameters.
    """

    def __init__(self, n_components: int, n_iters= 300, tol = 0.001, seed=0):
        self.n_components = n_components
        self.n_iters = n_iters
        self.tol = tol
        self.seed = seed

    def init_parameter (self, X):
        kmeans = KMeans(n_clusters=self.n_components, n_init=1, random_state=self.seed).fit(X)
        labels = kmeans.labels_

        n_row, n_col = X.shape
        self.resp = np.zeros((n_row, self.n_components))
        self.means = np.zeros((self.n_components, n_col))
        self.covs = np.zeros((self.n_components, n_col, n_col))
        self.weights = np.ones(self.n_components)
        
        for k in range(self.n_components):
            mask = np.where(labels == k)
            self.means[k, :] = np.mean(X[mask], axis = 0)
            x = X[mask] - self.means[k, :]
            self.covs[k, :, :] = np.dot(self.weights[k] * x.T, x) / X[mask].shape[0]      

    def init_parameters_rand (self, X):
        # data's dimensionality and responsibility vector
        n_row, n_col = X.shape     
        self.resp = np.zeros((n_row, self.n_components))

        # initialize parameters
        np.random.seed(self.seed)
        chosen = np.random.choice(n_row, self.n_components, replace = False)
        self.means = X[chosen]
        self.weights = np.full(self.n_components, 1 / self.n_components)
        
        # for np.cov, rowvar = False, 
        # indicates that the rows represents obervation
        shape = self.n_components, n_col, n_col
        self.covs = np.full(shape, np.cov(X, rowvar = False))

    def fit(self, X):

        self.init_parameters_rand(X)

        log_likelihood = 0
        self.converged = False
        self.log_likelihood_trace = []      

        for i in range(self.n_iters):
            log_likelihood_new = self._do_estep(X)
            self._do_mstep(X)

            if abs(log_likelihood_new - log_likelihood) <= self.tol:
                self.converged = True
                print(i)
                break
  
            log_likelihood = log_likelihood_new
            self.log_likelihood_trace.append(log_likelihood)
        
        if (not self.converged):
            print(self.n_iters)

        return self

    def _do_estep(self, X):
        """
        E-step: compute responsibilities,
        update resp matrix so that resp[j, k] is the responsibility of cluster k for data point j,
        to compute likelihood of seeing data point j given cluster k, use multivariate_normal.pdf
        """
        self._compute_log_likelihood(X)
        log_likelihood = np.sum(np.log(np.sum(self.resp, axis = 1)))

        # normalize over all possible cluster assignments
        self.resp = self.resp / self.resp.sum(axis = 1, keepdims = 1)
        return log_likelihood

    def _compute_log_likelihood(self, X):
        for k in range(self.n_components):
            prior = self.weights[k]
            likelihood = multivariate_normal(self.means[k], self.covs[k], allow_singular=True).pdf(X)
            self.resp[:, k] = prior * likelihood

        return self

    def _do_mstep(self, X):
        """M-step, update parameters"""

        # total responsibility assigned to each cluster, N^{soft}
        resp_weights = self.resp.sum(axis = 0)
        
        # weights
        self.weights = resp_weights / X.shape[0]

        # means
        weighted_sum = np.dot(self.resp.T, X)
        self.means = weighted_sum / resp_weights.reshape(-1, 1)
        # covariance
        for k in range(self.n_components):
            diff = (X - self.means[k]).T
            weighted_sum = np.dot(self.resp[:, k] * diff, diff.T)
            self.covs[k] = weighted_sum / resp_weights[k]

        return self

    def predict(self, X):
        n, dim = X.shape
        test_data = np.asarray(X)
        test_num_points = n
        test_z = np.zeros((test_num_points, self.n_components))

        for k in range(self.n_components):
        
            prior = self.weights[k]
            likelihood = multivariate_normal(self.means[k], self.covs[k], allow_singular=True).pdf(X)
            test_z[:, k] = prior * likelihood

        test_z =  test_z /  test_z.sum(axis = 1, keepdims = 1)
        
        output = []
        for row in test_z:
            output.append(np.argmax(row))
        
        return output

In [131]:
import pandas as pd
df = pd.read_csv('urbanGB.all/urbanGB.txt', header=None, names=['x', 'y'])
df_label = pd.read_csv('urbanGB.all/urbanGB.labels.txt', header=None, names=['label'])
df = df.join(df_label)
df = df.sample(150000, random_state=0).reset_index(drop=True)


In [132]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standard_df = scaler.fit_transform(df.drop(['label'], axis=1))

In [133]:
from sklearn.mixture import GaussianMixture
import sklearn.metrics
gm = GaussianMixture(n_components=5, random_state=0)
gm.fit(standard_df)
gm.n_iter_


5

In [134]:
sklrn_output = gm.predict(standard_df)


In [135]:
sklearn.metrics.silhouette_score(standard_df, sklrn_output)

0.49660397996577677

In [136]:
gmm = GMM(n_components=5)
gmm.fit(standard_df)

77


In [137]:
my_output = gmm.predict(standard_df)

In [138]:
sklearn.metrics.silhouette_score(standard_df, my_output)

0.2774864844608824